In [1]:
import sys
sys.path.append("..")

In [2]:
from functools import wraps
from os.path import join, isfile
import pandas as pd

from src.config import *
from src.db import load_from_local, write_local

import logging
logging.basicConfig(level=logging.DEBUG)

In [51]:
def checking_file(table_name):
    return isfile(join(SaveConfig.DATA_DIR, f"{table_name}.csv"))

def load_cache(func):
    """cache to local csv"""
    @wraps(func)
    def wrapper(table_name):
        if not checking_file(table_name):
            table = func(table_name)
            write_local(table_name, data=table, is_out=False)
            return table
        else:
            return load_from_local(table_name)
    return wrapper

@load_cache
def load_from_mysql(table_name: str):
    """读取远程mysql数据表"""
    logging.debug(msg=f"Reading mysql table {table_name}")
    table = pd.read_sql_table(con=RemoteMySQLConfig.engine, table_name=f"{table_name}")
    return table

In [52]:
table = load_from_mysql('i_queue_io')

DEBUG:root:Reading mysql table i_queue_io
INFO:root:csv write table i_queue_io succeed!
